<center><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks">unSkript Runbooks<a class="jp-InternalAnchorLink" href="#unSkript-Runbooks" target="_self">&para;</a></h1>
<div class="alert alert-block alert-success">
<h3 id="Objective"><strong>Objective</strong><a class="jp-InternalAnchorLink" href="#Objective" target="_self">&para;</a></h3>
<strong>Delete unused NAT gateways using unSkript actions.</strong></div>
</center><center>
<h2 id="Delete-Unused-NAT-Gateways">Delete Unused NAT Gateways<a class="jp-InternalAnchorLink" href="#Delete-Unused-NAT-Gateways" target="_self">&para;</a></h2>
</center>
<h1 id="Steps-Overview">Steps Overview<a class="jp-InternalAnchorLink" href="#Steps-Overview" target="_self">&para;</a></h1>
<p>1. AWS Find Unused NAT Gateways<br>2. AWS Delete NAT Gateway</p>

In [5]:
if nat_gateway_ids and not region:
    raise SystemExit("Enter region for given NAT gateways!")
if region == None:
    region = ""

<h3 id="Filter-Unattached-EBS-Volumes">AWS Find Unused NAT Gateways</h3>
<p>Here we will use unSkript&nbsp;<strong>AWS Find Unused NAT Gateways</strong> action. This action filters all the NAT Gateways from the given region and returns a list of all the unused NAT Gateways. It will execute if the nat_gateway_ids&nbsp;parameter is not passed.</p>
<blockquote>
<p>Input parameters: <code>region</code></p>
</blockquote>
<blockquote>
<p>Output variable: <code>unused_nat_gateways</code></p>
</blockquote>

In [ ]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import Optional, Tuple
from pydantic import BaseModel, Field
from datetime import datetime, timedelta
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions
import pprint

from beartype import beartype
@beartype
def aws_filter_unused_nat_gateway_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def is_nat_gateway_used(handle, nat_gateway, start_time, end_time,number_of_days):
    datapoints = []
    if nat_gateway['State'] != 'deleted':
        # Get the metrics data for the specified NAT Gateway over the last 7 days
        metrics_data = handle.get_metric_statistics(
            Namespace='AWS/NATGateway',
            MetricName='ActiveConnectionCount',
            Dimensions=[
                {
                    'Name': 'NatGatewayId',
                    'Value': nat_gateway['NatGatewayId']
                },
            ],
            StartTime=start_time,
            EndTime=end_time,
            Period=86400*number_of_days,
            Statistics=['Sum']
        )
        datapoints += metrics_data['Datapoints']
    if len(datapoints) == 0 or metrics_data['Datapoints'][0]['Sum']==0:
        return False
    else:
        return True


@beartype
def aws_filter_unused_nat_gateway(handle, number_of_days: int = 7, region: str = "") -> Tuple:
    """aws_get_natgateway_by_vpc Returns an array of NAT gateways.

        :type region: string
        :param region: Region to filter NAT Gateways.

        :type number_of_days: int
        :param number_of_days: Number of days to check the Datapoints.

        :rtype: Array of NAT gateways.
    """
    result = []
    end_time = datetime.utcnow()
    start_time = end_time - timedelta(days=number_of_days)
    all_regions = [region]
    if not region:
        all_regions = aws_list_all_regions(handle)

    for reg in all_regions:
        try:
            ec2Client = handle.client('ec2', region_name=reg)
            cloudwatch = handle.client('cloudwatch', region_name=reg)
            response = ec2Client.describe_nat_gateways()
            for nat_gateway in response['NatGateways']:
                nat_gateway_info = {}
                if not is_nat_gateway_used(cloudwatch, nat_gateway, start_time, end_time,number_of_days):
                    nat_gateway_info["nat_gateway_id"] = nat_gateway['NatGatewayId']
                    nat_gateway_info["region"] = reg
                    result.append(nat_gateway_info)
        except Exception as e:
            pass

    if len(result) != 0:
        return (False, result)
    else:
        return (True, None)



task = Task(Workflow())
task.configure(inputParamsJson='''{
    "region": "region",
    "number_of_days": "int(number_of_days)"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not nat_gateway_ids",
    "condition_result": true
    }''')
task.configure(outputName="unused_nat_gateways")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_filter_unused_nat_gateway, lego_printer=aws_filter_unused_nat_gateway_printer, hdl=hdl, args=args)

<h3 id="Modify-Snapshot-Output">Modify Unused NAT Gateways Output</h3>
<p>In this action, we modify the output from Step 1 and return a list of unused NAT gateways.&nbsp;</p>
<blockquote>
<p>Output variable: nat_gateways</p>
</blockquote>

In [24]:
nat_gateways = []
try:
    if unused_nat_gateways[0] == False:
        for nat in unused_nat_gateways[1]:
            nat_gateways.append(nat)
except Exception as e:
    if nat_gateway_ids:
        for nat in nat_gateway_ids:
            nat_ids = {}
            nat_ids["nat_gateway_id"] = nat
            nat_ids["region"] = region
            nat_gateways.append(nat_ids)
    else:
        raise Exception(e)

<h3 id="Delete-EBS-Volume">AWS Delete NAT Gateway</h3>
<p>Here we will use the unSkript <strong>AWS Delete NAT Gateway</strong> action. In this action, we will pass the list of unused NAT Gateways from Step 1 and delete those NAT Gateways.</p>
<blockquote>
<p>Input parameters: <code>nat_gateway_id</code>, <code>region</code></p>
</blockquote>
<blockquote>
<p>Output variable: <code>delete_status</code></p>
</blockquote>

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Dict
import pprint


from beartype import beartype
@beartype
def aws_delete_nat_gateway_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_delete_nat_gateway(handle, nat_gateway_id: str, region: str) -> Dict:
    """aws_delete_nat_gateway Returns an dict of NAT gateways information.

        :type region: string
        :param region: AWS Region.

        :type nat_gateway_id: string
        :param nat_gateway_id: ID of the NAT Gateway.

        :rtype: dict of NAT gateways information.
    """
    try:
        ec2Client = handle.client('ec2', region_name=region)
        response = ec2Client.delete_nat_gateway(NatGatewayId=nat_gateway_id)
        return response
    except Exception as e:
        raise Exception(e)


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "nat_gateway_id": "iter.get(\\"nat_gateway_id\\")",
    "region": "iter.get(\\"region\\")"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "nat_gateways",
    "iter_parameter": ["nat_gateway_id","region"]
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(nat_gateways) != 0",
    "condition_result": true
    }''')

task.configure(outputName="delete_status")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_delete_nat_gateway, lego_printer=aws_delete_nat_gateway_printer, hdl=hdl, args=args)

### Conclusion
<p>In this Runbook, we demonstrated the use of unSkript's AWS actions to filter unused NAT Gateways and delete those. To view the full platform capabilities of unSkript please visit <a href="https://us.app.unskript.io" target="_blank" rel="noopener">https://us.app.unskript.io</a></p>